# A/B Testing in PyMC: part 1

<div class="alert alert-warning">
<h2>Goal of this session:</h2>

In this hands-on session, you will continue working with the synthetic dataset from yesterday. Your goal is to be able to perform Bayesian inference using the PyMC library. The insight gained from this session will help you understand:

- How to perform Bayesian A/B testing on data with Bernoulli conversions
- How to interpret the results
- How to use ArviZ library to visualize the results

</div>


Let's practice what you have learned this morning. You will be using the [PyMC library](https://www.pymc.io/welcome.html) to do Bayesian inference and explore posterior distributions. 

You only workout the case of two variants A and B with **Bernoulli conversions** using the data from yesterday. Let's start by importing the necessary libraries.

In [ ]:
# Load libraries
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import numpy as np
import pymc as pm

## Your First Try

Remember the example with synthetic data where we had a website with two variants A and B. Let's assume that we have a Bernoulli likelihood for each variant, and we can model the conversion rate as Beta distribution for each variant. To start with, let's assume that we have identical priors for both variants with $\alpha = 1$ and $\beta = 1$:

In [ ]:
# let's define the variants A and B
variants  = ['A', 'B']

# let's define parameters for a weak prior for the conversion rates
prior_alpha, prior_beta = [1, 1]

Now, replace the `< >` with the values observed from the data:

In [ ]:
# number of visitors for each variant 
visitors   = [< >, < >]

# number of conversions (sign ups) for each variant
conversion = [< >, < >]

The model, defined in the next cell, consist of four main components:
- Prior
- Likelihood
- Uplift (difference between A and B)
- Posterior

Let's run the code

In [ ]:
with pm.Model() as example_model:

    # Priors for unknown model parameters
    theta = pm.Beta("theta", 
                    alpha = prior_alpha, 
                    beta  = prior_beta, 
                    shape = 2)
    
    # Likelihood (sampling distribution) of observations
    obs = pm.Binomial("y", 
                      n        = visitors, 
                      observed = conversion, 
                      p        = theta,
                      shape    = 2) 
    
    # Difference between variants
    relative_uplift = pm.Deterministic("uplift", 
                                        theta[1] / theta[0] - 1)

    # Draw samples from the prior
    trace = pm.sample(draws=1000, return_inferencedata=False)

Note that in the last line we set `return_inferencedata=False` and you can set to True later on.

Run each of the following cells and try to understand their output.

In [ ]:
# Check the unobserved random variables:
example_model.unobserved_RVs    

In [ ]:
# Check the observed (synthetic) random variable:
example_model.observed_RVs    

In [ ]:
# Check the deterministic random variable:
example_model.deterministics

In [ ]:
# check the trace, i.e. posterior distribution of the conversion rates
trace

In [ ]:
# find out the shape of the trace
trace.theta.shape,  trace.uplift.shape 

<div class="alert alert-info">
<h4>Task 1</h4>

1. What do you learn from the `shapes` of the converation rates and the uplift?

2. Why do you have 4000 samples despite setting `draws=1000`?

</div>

write your answer here

Let's continue by exploring the posterior distributions of the conversion rates and the uplift.

In [ ]:
# let's create a dataframe with the output of the posterior distribution
df = pd.concat([pd.DataFrame(trace['theta']), 
                pd.DataFrame(trace['uplift'])],
                axis=1)

df.columns = ['theta_A','theta_B','uplift']
df.head(3)

<div class="alert alert-info">
<h4>Task 2</h4>

1. What are the average conversion rates according to the posterior distributions? Are they inline with your expectations? 

2. Are the averages closer to the conversion rates observed from data or the priors? 

3. What is the average uplift. Discuss what you observe.

</div>

write your answer here

## PyMC Visualisation Options

Let's redo the previous example but this time we set the `return_inferencedata` to True. This will return an `InferenceData` object which we can use to visualise the posterior distributions. The `InferenceData` object contains the posterior samples and many other details about the model.

In [ ]:
with pm.Model() as example_model:

    # Priors for unknown model parameters
    theta = pm.Beta("theta", 
                    alpha = prior_alpha, 
                    beta  = prior_beta, 
                    shape = 2)
    
    # Likelihood (sampling distribution) of observations
    obs = pm.Binomial("y", 
                      n        = visitors, 
                      observed = conversion, 
                      p        = theta,
                      shape    = 2) 
    
    # Difference between variants
    relative_uplift = pm.Deterministic("uplift", 
                                        theta[1] / theta[0] - 1)

    # Draw samples from the prior
    trace = pm.sample(draws=1000, return_inferencedata=True)

In [ ]:
# let's check the trace
trace

<div class="alert alert-info">
<h4>Task 3</h4>

In the above result, click on the **posterior** tab:

1. What do you see under the **Data variables** section?

2. Click on the database icon at the right-end of the **theta** row. What do you see?

3. What do you see under the **Dimensions** section?

</div>

write your answer here

We can also use the ArviZ library for plotting. The main advantage is that it works pretty well with the output data. Also, it gives the HDI for the relative Uplift distribution.

In [ ]:
az.plot_posterior(trace.posterior["uplift"], figsize=(5, 2),textsize=8,
                        point_estimate=None,
                        ref_val=0,  
                        hdi_prob= 0.95, #'hide', 
                        rope=[-0.02, 0.02]
                )
plt.title("B vs. A Uplift Distribution", fontsize=8,)
plt.axvline(x=0, color='#87ceeb', linestyle='--', linewidth=1);

<div class="alert alert-info">
<h4>Task 4</h4>

1. What is your interpretation of the 95% HDI?

2. What is the probability that the uplift is greater than 0?
    
3. What do you learn from the ROPE?

4. What is your conclusion about the A/B test?

</div>

write your answer here

There are many other plots that you can do with ArviZ. You can find more examples [here](https://arviz-devs.github.io/arviz/examples/index.html), but let's try a few of them.

In [ ]:
# what about the theta distributions?
az.plot_forest(trace, var_names=["theta"], combined=True, kind="ridgeplot", ridgeplot_alpha=0.1);

With `plot_trace` we get a full picture.

In [ ]:
az.plot_trace(trace, compact=True, combined=True)
plt.tight_layout();

Yet another view with `plot_pair` and `plot_violin`:

In [ ]:
az.plot_pair(trace, var_names=["theta"], kind="kde", marginals=True, figsize=(5, 5));

In [ ]:
az.plot_violin(trace, var_names=["theta"], figsize=(7, 4));

<div class="alert alert-info">
<h4>Task 5</h4>


Try an informative prior for $\theta_A$ and $\theta_B$ especially one that is skewed, and re-do what you have done so far. How does that affect your conclusion about the difference between the two variants?

</div>

write your answer here